### Считывание данных и загрузка в postgres, объединение данных в датафрейм

### Используемые таблицы с geonames:


- admin1CodesASCII
- cities15000
- countryInfo

- таблицы geonames можно скачать здесь http://download.geonames.org/export/dump/
- Тестовый датасет: https://disk.yandex.ru/d/wC296Rj3Yso2AQ


### Данные по geonames содержат следущую информацию

- geonameid: идентификатор записи в базе данных
- name: название георафической точки (в формате utf8)
- asiiname: название георафических точек в обычных символах ASII
- alternatenames: альтернативные названия, через запятую, автоматически транслитерированные имена в ASII
- latitude: широта в десятичных градусах
- longitude: долгота в десятичных градусах
- feature class: классы гео сущностей (населенный пункт и т.д.)
- feature code: коды гео сущностей
- country code: ISO-3166 двухбуквенный код страны
- cc2: альтернативные коды страны, разделенные запятой, ISO-3166 двухбуквенный код страны
- admin1 code: код административной единицы
- admin2 code: код второго административного деления, округ в США
- admin3 code: код третьего уровня административного деления
- admin4 code: код четвертого уровня административного деления
- population: население
- elevation: высота, в метрах
- dem: цифровая модель рельефа
- timezone: идентификатор временной зоны iana
- modification_date: дата последнего изменения в формате yyy-MM-dd

In [1]:
import os
import pandas as pd
import psycopg2
import sqlalchemy

from sqlalchemy import create_engine, text
from sqlalchemy.engine.url import URL
from sqlalchemy import select
from pathlib import Path

In [2]:
# подключаемся к СУБД
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres', 
    'password': '***', 
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'query': {}
}  

engine = create_engine(URL(**DATABASE))

C:\Users\User\AppData\Local\Temp\ipykernel_8240\28999779.py:12: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  engine = create_engine(URL(**DATABASE))


In [3]:
Countries_dop = ['RU', 'BY', 'KG', 'KZ', 'AM', 'GE', 'RS', 'ME']
model = Path('_'.join(Countries_dop))

## Загрузка данных в СУБД

#### Города

In [4]:
cities = pd.read_csv('http://download.geonames.org/export/dump/cities15000.zip',\
                     sep = '\t', header = None, low_memory=False,\
                     names = ['geonameid', 'name', 'ascii_name', 'alternate_names',\
                              'latitude', 'longitude', 'feature_class', 'feature_code',\
                              'country_code', 'cc2', 'admin1_code', 'admin2_code',\
                              'admin3_code', 'admin4_code', 'population', 'elevation',\
                              'dem', 'timezone', 'modification_date'],
                    usecols = ['geonameid', 'name', 'alternate_names', 'country_code',\
                               'admin1_code', 'population']).dropna()

cities['code'] = cities.country_code +'.'+ cities.admin1_code
cities = cities.drop('admin1_code', axis=1)


cities = cities[cities.country_code.isin(Countries_dop)]
cities.head()

,geonameid,name,alternate_names,country_code,population,code
94,174875,Kapan,"Ghap'an,Ghapan,Ghap’an,Kafan,Kafin,Kapan,Kapan...",AM,33160,AM.08
95,174895,Goris,"Geryusy,Goris,Горис,Գորիս",AM,20379,AM.08
96,174972,Hats’avan,"Acavan,Atsavan,Hats'avan,Hats’avan,Sisian,Ацав...",AM,15208,AM.08
97,174979,Artashat,"Artachat,Artasat,Artasatas,Artasato,Artaschat,...",AM,20562,AM.02
98,174991,Ararat,"Ararat,Araratas,Ararato,Davalinskiy Tsemzavod,...",AM,28832,AM.02


In [5]:
cities.alternate_names = cities.alternate_names.str.split(',')
cities = cities.explode('alternate_names')
cities = cities[cities.name!=cities.alternate_names]
cities = cities.drop_duplicates(subset=['name', 'alternate_names'])
cities

,geonameid,name,alternate_names,country_code,population,code
94,174875,Kapan,Ghap'an,AM,33160,AM.08
94,174875,Kapan,Ghapan,AM,33160,AM.08
94,174875,Kapan,Ghap’an,AM,33160,AM.08
94,174875,Kapan,Kafan,AM,33160,AM.08
94,174875,Kapan,Kafin,AM,33160,AM.08
...,...,...,...,...,...,...
21263,8521440,Dzerzhinsky,Дзержинский,RU,47163,RU.47
21265,11886891,Fedorovskiy,Fedorovskij,RU,23375,RU.32
21265,11886891,Fedorovskiy,Федоровский,RU,23375,RU.32
21266,12041452,Mezgor'e,Mezhgor'e,RU,15861,RU.08


In [6]:
cities.to_sql('cities', con=engine, index=False, if_exists='replace')

40

#### Области и провинции

In [7]:
admin = pd.read_csv('http://download.geonames.org/export/dump/admin1CodesASCII.txt',\
                     sep = '\t', low_memory=False, header = None,\
                    names = ['code', 'region', 'ascii_region', 'geonameid'],\
                   usecols = ['code', 'region', 'geonameid'])
admin['country_code'] = admin.code.str.split('.').str[0]
admin = admin[admin.country_code.isin(Countries_dop)]
admin

,code,region,geonameid,country_code
82,AM.02,Ararat,409313,AM
83,AM.08,Syunik,409314,AM
84,AM.10,Vayots Dzor,409315,AM
85,AM.11,Yerevan,616051,AM
86,AM.01,Aragatsotn,828259,AM
...,...,...,...,...
2835,RU.64,Sakhalin Oblast,2121529,RU
2836,RU.44,Magadan Oblast,2123627,RU
2837,RU.92,Kamchatka,2125072,RU
2838,RU.15,Chukotka,2126099,RU


In [8]:
admin.to_sql('admin', con=engine, index=False, if_exists='replace')

170

#### Страны

In [9]:
country = pd.read_csv('http://download.geonames.org/export/dump/countryInfo.txt',\
                           sep='\t', comment='#', low_memory=False, header = None, \
                          names = ['country_code', 'iso_3', 'iso_numeric', 'fips_code',\
                              'country', 'capital', 'area', 'population',\
                              'continent', 'tld', 'currency_code', 'currency_name', 'phone',\
                                   'postal_code_format', 'postal_code_regex', 'languages',\
                                   'geonameid', 'neighbours', 'equivalent_fips_code'],
                          usecols = [
                              'geonameid', 'country_code', 'country',
                          ])
country

,country_code,country,geonameid
0,AD,Andorra,NaN
1,AE,United Arab Emirates,290557.0
2,AF,Afghanistan,1149361.0
3,AG,Antigua and Barbuda,3576396.0
4,AI,Anguilla,3573511.0
...,...,...,...
247,ZA,South Africa,NaN
248,ZM,Zambia,NaN
249,ZW,Zimbabwe,878675.0
250,CS,Serbia and Montenegro,NaN


In [10]:
country.to_sql('country', con=engine, index=False, if_exists='replace')

252

### Запросы к базе

In [11]:
country = pd.read_sql_query('select * from "country"',con=engine)
admin = pd.read_sql_query('select * from "admin"',con=engine)
cities = pd.read_sql_query('select * from "cities"',con=engine)

## Объединение данных

In [12]:
def new_name(df):
    return (pd
           .concat([df,
                   df.drop_duplicates(subset='geonameid').assign(alternate_name=df.name)])
           .drop_duplicates()
           .reset_index(drop=True)
            )

In [13]:
df_cities = (cities
            .merge(admin[['code', 'region']], on = 'code', how = 'left')
            .merge(country[['country_code', 'country']], on = 'country_code', how = 'left')
            .assign(geo_type='city')
            .drop_duplicates()
            .reset_index(drop=True)
            )
df_cities = new_name(df_cities)
df_cities

,geonameid,name,alternate_names,country_code,population,code,region,country,geo_type,alternate_name
0,174875,Kapan,Ghap'an,AM,33160,AM.08,Syunik,Armenia,city,NaN
1,174875,Kapan,Ghapan,AM,33160,AM.08,Syunik,Armenia,city,NaN
2,174875,Kapan,Ghap’an,AM,33160,AM.08,Syunik,Armenia,city,NaN
3,174875,Kapan,Kafan,AM,33160,AM.08,Syunik,Armenia,city,NaN
4,174875,Kapan,Kafin,AM,33160,AM.08,Syunik,Armenia,city,NaN
...,...,...,...,...,...,...,...,...,...,...
20378,8504965,Sampsonievskiy,Sampsonievskij,RU,39318,RU.42,Leningradskaya Oblast',Russia,city,Sampsonievskiy
20379,8505053,Vostochnoe Degunino,Восточное Дегунино,RU,95000,RU.48,Moscow,Russia,city,Vostochnoe Degunino
20380,8521440,Dzerzhinsky,Dzerzhinskij,RU,47163,RU.47,Moscow Oblast,Russia,city,Dzerzhinsky
20381,11886891,Fedorovskiy,Fedorovskij,RU,23375,RU.32,Khanty-Mansia,Russia,city,Fedorovskiy


In [14]:
df_region = (admin
            .merge(country[['country_code', 'country']], on = 'country_code', how='left')
            .rename(columns={'region':'name'})
            .assign(geo_type='region')
            .drop_duplicates()
            .reset_index(drop=True)
            )
df_region = new_name(df_region)
df_region

,code,name,geonameid,country_code,country,geo_type,alternate_name
0,AM.02,Ararat,409313,AM,Armenia,region,NaN
1,AM.08,Syunik,409314,AM,Armenia,region,NaN
2,AM.10,Vayots Dzor,409315,AM,Armenia,region,NaN
3,AM.11,Yerevan,616051,AM,Armenia,region,NaN
4,AM.01,Aragatsotn,828259,AM,Armenia,region,NaN
...,...,...,...,...,...,...,...
335,RU.64,Sakhalin Oblast,2121529,RU,Russia,region,Sakhalin Oblast
336,RU.44,Magadan Oblast,2123627,RU,Russia,region,Magadan Oblast
337,RU.92,Kamchatka,2125072,RU,Russia,region,Kamchatka
338,RU.15,Chukotka,2126099,RU,Russia,region,Chukotka


In [15]:
data = pd.concat([country, df_region, df_cities], axis=0)
data

,country_code,country,geonameid,code,name,geo_type,alternate_name,alternate_names,population,region
0,AD,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AE,United Arab Emirates,290557.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AF,Afghanistan,1149361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AG,Antigua and Barbuda,3576396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AI,Anguilla,3573511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20378,RU,Russia,8504965.0,RU.42,Sampsonievskiy,city,Sampsonievskiy,Sampsonievskij,39318.0,Leningradskaya Oblast'
20379,RU,Russia,8505053.0,RU.48,Vostochnoe Degunino,city,Vostochnoe Degunino,Восточное Дегунино,95000.0,Moscow
20380,RU,Russia,8521440.0,RU.47,Dzerzhinsky,city,Dzerzhinsky,Dzerzhinskij,47163.0,Moscow Oblast
20381,RU,Russia,11886891.0,RU.32,Fedorovskiy,city,Fedorovskiy,Fedorovskij,23375.0,Khanty-Mansia


In [16]:
data['country'] = data['country'].fillna(value=data['name'])
data['region'] = data['region'].fillna(value=data['name'])
data['alternate_names'] = data['alternate_names'].fillna(value=data['name'])
data['name'] = data['name'].fillna(value=data['alternate_names'])
data = data.drop_duplicates()
data = data[data.country_code.isin(Countries_dop)].reset_index(drop=True)
data

,country_code,country,geonameid,code,name,geo_type,alternate_name,alternate_names,population,region
0,AM,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BY,Belarus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GE,Georgia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KG,Kyrgyzstan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KZ,Kazakhstan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20726,RU,Russia,8504965.0,RU.42,Sampsonievskiy,city,Sampsonievskiy,Sampsonievskij,39318.0,Leningradskaya Oblast'
20727,RU,Russia,8505053.0,RU.48,Vostochnoe Degunino,city,Vostochnoe Degunino,Восточное Дегунино,95000.0,Moscow
20728,RU,Russia,8521440.0,RU.47,Dzerzhinsky,city,Dzerzhinsky,Dzerzhinskij,47163.0,Moscow Oblast
20729,RU,Russia,11886891.0,RU.32,Fedorovskiy,city,Fedorovskiy,Fedorovskij,23375.0,Khanty-Mansia


In [17]:
data = data.drop('alternate_name', axis=1)

### Запишем объединенный датасет

In [18]:
if not os.path.exists(model):
    os.makedirs(model)
data.to_csv(model/'data.csv', index=False)